In [1]:
#coding:utf-8

# 必要なモジュールをインポート
import codecs
import corenlp
import csv
import nltk
import pandas as pd
import pydot
import numpy as np
import re
import subprocess
from stemming.porter2 import stem
import xml.etree.ElementTree as ET

### Anaconda環境に新しくインストールするもの
#### Starnford Corenlp
基本はここ => http://qiita.com/yubessy/items/1869ac2c66f4e76cd6c5
<br>ただし、pipを使う時に、anaconda環境のpipでインストールする。
例えば、以下のようなコマンド。
```
.pyenv/versions/anaconda2-2.5.0/bin/pip install corenlp-python
```
#### stem
ここからダウンロード => https://pypi.python.org/pypi/stemming/1.0
<br>ディレクトリに行って、以下のコマンド。
```
python setupy.py install
```
#### pydot
以下のコマンドで入る。
```
conda install -c anaconda pydot=1.0.28
```

In [2]:
# まずデータをダウンロード
f = open("./nlp.txt","rb")
nlp_data = f.read()

In [3]:
nlp_data

'Natural language processing\nFrom Wikipedia, the free encyclopedia\n\nNatural language processing (NLP) is a field of computer science, artificial intelligence, and linguistics concerned with the interactions between computers and human (natural) languages. As such, NLP is related to the area of humani-computer interaction. Many challenges in NLP involve natural language understanding, that is, enabling computers to derive meaning from human or natural language input, and others involve natural language generation.\n\nHistory\n\nThe history of NLP generally starts in the 1950s, although work can be found from earlier periods. In 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence.\n\nThe Georgetown experiment in 1954 involved fully automatic translation of more than sixty Russian sentences into English. The authors claimed that within three or five years, machine translati

## 50. 文区切り
(. or ; or : or ? or !) → 空白文字 → 英大文字というパターンを文の区切りと見なし，入力された文書を1行1文の形式で出力せよ．

#### [方針]
正規表現を使ってパターン分割する。

In [4]:
pattern = r'\.\s[A-Z]|\;\s[A-Z]|\:\s[A-Z]|\?\s[A-Z]|\!\s[A-Z]'
sentences = re.finditer(pattern, nlp_data)

In [7]:
index_list=[]
sentence_list = []
for i,so in enumerate(sentences):
	if i == 0:
		_0_index = so.end()-1
		sentence_list.extend([nlp_data[:_0_index].rstrip()])

		print nlp_data[:_0_index].rstrip()
		index_list.extend([_0_index])

	else:
		start_index = index_list[i-1]
		end_index = so.end()-1
		sentence_list.extend([nlp_data[start_index:end_index]])

		print nlp_data[start_index:end_index]
		index_list.extend([end_index])

Natural language processing
From Wikipedia, the free encyclopedia

Natural language processing (NLP) is a field of computer science, artificial intelligence, and linguistics concerned with the interactions between computers and human (natural) languages.
As such, NLP is related to the area of humani-computer interaction. 
Many challenges in NLP involve natural language understanding, that is, enabling computers to derive meaning from human or natural language input, and others involve natural language generation.

History

The history of NLP generally starts in the 1950s, although work can be found from earlier periods. 
In 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence.

The Georgetown experiment in 1954 involved fully automatic translation of more than sixty Russian sentences into English. 
The authors claimed that within three or five years, machine translation woul

#### [解説]
(参照) http://kenichia.hatenablog.com/entry/2016/02/15/174813

* 正規表現の意味
    ```
    pattern = r'\.\s[A-Z]|\;\s[A-Z]|\:\s[A-Z]|\?\s[A-Z]|\!\s[A-Z]'
    ```
    \ => ここから始まり<br>
	\s => スペース(空白文字)<br>
	[A-Z] => 英大文字<br>
	| => または<br>

	つまり、".などの記号+スペース+英大文字"の組み合わせをorで4種類作っているということ<br>
	若干これは冗長な気がする。記号の部分をひとつにまとめられたらきっともっとすっきりするはず。


* 正規表現で文字列を探すとき
	(参照)http://qiita.com/wanwanland/items/ce272419dde2f95cdabc
		- re.split()は、patternを削除して分割する
		- re.match/search/findall/finditer とかの使い分けが大事

## 51. 単語の切り出し
	空白を単語の区切りとみなし，50の出力を入力として受け取り，1行1単語の形式で出力せよ．ただし，文の終端では空行を出力せよ．

In [11]:
word_list = []
for sentence in sentence_list:
	for word in re.split(" ", sentence):     # 空白で分割
		if word[:-1] == ".":                 # インデックスの"-1"は、一番後ろを指す
			print word,"\n"
			eos_word = word+"\n"
			word_list.extend([eos_word])
		else:
			print word
			word_list.extend([word])

Natural
language
processing
From
Wikipedia,
the
free
encyclopedia

Natural
language
processing
(NLP)
is
a
field
of
computer
science,
artificial
intelligence,
and
linguistics
concerned
with
the
interactions
between
computers
and
human
(natural)
languages.
As
such,
NLP
is
related
to
the
area
of
humani-computer
interaction.

Many
challenges
in
NLP
involve
natural
language
understanding,
that
is,
enabling
computers
to
derive
meaning
from
human
or
natural
language
input,
and
others
involve
natural
language
generation.

History

The
history
of
NLP
generally
starts
in
the
1950s,
although
work
can
be
found
from
earlier
periods.

In
1950,
Alan
Turing
published
an
article
titled
"Computing
Machinery
and
Intelligence"
which
proposed
what
is
now
called
the
Turing
test
as
a
criterion
of
intelligence.

The
Georgetown
experiment
in
1954
involved
fully
automatic
translation
of
more
than
sixty
Russian
sentences
into
English.

The
authors
claimed
that
within
three
or
five
years,
machine
translation
woul

## 52. ステミング
	51の出力を入力として受け取り，Porterのステミングアルゴリズムを適用し，単語と語幹をタブ区切り形式で出力せよ．
    Pythonでは，Porterのステミングアルゴリズムの実装としてstemmingモジュールを利用するとよい．

* stemのインストール =>
(参照) https://pypi.python.org/pypi/stemming/1.0
<br> ダウンロードして、"python setup.py install"を叩く！！

In [12]:
for word in word_list:
	print word,"\t",stem(word)

Natural 	Natur
language 	languag
processing
From 	processing
From
Wikipedia, 	Wikipedia,
the 	the
free 	free
encyclopedia

Natural 	encyclopedia

Natur
language 	languag
processing 	process
(NLP) 	(NLP)
is 	is
a 	a
field 	field
of 	of
computer 	comput
science, 	science,
artificial 	artifici
intelligence, 	intelligence,
and 	and
linguistics 	linguist
concerned 	concern
with 	with
the 	the
interactions 	interact
between 	between
computers 	comput
and 	and
human 	human
(natural) 	(natural)
languages. 	languages.
As 	As
such, 	such,
NLP 	NLP
is 	is
related 	relat
to 	to
the 	the
area 	area
of 	of
humani-computer 	humani-comput
interaction. 	interaction.
 	
Many 	Mani
challenges 	challeng
in 	in
NLP 	NLP
involve 	involv
natural 	natur
language 	languag
understanding, 	understanding,
that 	that
is, 	is,
enabling 	enabl
computers 	comput
to 	to
derive 	deriv
meaning 	mean
from 	from
human 	human
or 	or
natural 	natur
language 	languag
input, 	input,
and 	and
others 	other
involve 	involv
natu

## 53. Tokenization
Stanford Core NLPを用い，入力テキストの解析結果をXML形式で得よ．また，このXMLファイルを読み込み，入力テキストを1行1単語の形式で出力せよ．

* 方針<br>
    corenlpはコマンドライン上で動かした方が早い気がするので、以下のjavaをコンパイルすると、自動でxmlファイルができます。(1分未満)
    <br>ただし、-file "input.txt"の部分を、インプットするファイル名に変える。
```
	java -cp "*" -Xmx2g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,ssplit,pos,lemma,ner,parse,dcoref -file input.txt
```
   
* 注意<br>
    corenlpのダウンロードするversionが古いと、エラー等で動かないことがある<br>
    (参照) http://qiita.com/yubessy/items/1869ac2c66f4e76cd6c5
	


In [13]:
xml_file_name = "./nlp.txt.xml"
tree = ET.parse(xml_file_name)
for word in tree.iter('word'):
	print word.text

Natural
language
processing
From
Wikipedia
,
the
free
encyclopedia
Natural
language
processing
-LRB-
NLP
-RRB-
is
a
field
of
computer
science
,
artificial
intelligence
,
and
linguistics
concerned
with
the
interactions
between
computers
and
human
-LRB-
natural
-RRB-
languages
.
As
such
,
NLP
is
related
to
the
area
of
humani-computer
interaction
.
Many
challenges
in
NLP
involve
natural
language
understanding
,
that
is
,
enabling
computers
to
derive
meaning
from
human
or
natural
language
input
,
and
others
involve
natural
language
generation
.
History
The
history
of
NLP
generally
starts
in
the
1950s
,
although
work
can
be
found
from
earlier
periods
.
In
1950
,
Alan
Turing
published
an
article
titled
``
Computing
Machinery
and
Intelligence
''
which
proposed
what
is
now
called
the
Turing
test
as
a
criterion
of
intelligence
.
The
Georgetown
experiment
in
1954
involved
fully
automatic
translation
of
more
than
sixty
Russian
sentences
into
English
.
The
authors
claimed
that
within
three
or
five

* その他<br>
    pythonで"subprocess"というモジュールを使うと、コマンドライン上での操作をpythonのコード内に挟むこともできる

## 54. 品詞タグ付け
	Stanford Core NLPの解析結果XMLを読み込み，単語，レンマ，品詞をタブ区切り形式で出力せよ．
	(参照) http://qiita.com/segavvy/items/4d55805352089332828e